## Prediction Jupyter Notebook For Direct Interactivity

In [ ]:
import pandas as pd
import joblib
import requests
from io import BytesIO

# GitHub raw file URLs
CSV_URL = "https://raw.githubusercontent.com/NdubuakuMiracle/housing-in-melbourne/main/data/processed/X_train.csv"
MODEL_URL = "https://raw.githubusercontent.com/NdubuakuMiracle/housing-in-melbourne/main/models/price_by_sub_area_lat_lon_model.pkl"

# Load dataset from GitHub
X_train = pd.read_csv(CSV_URL)

# Load model from GitHub
response = requests.get(MODEL_URL)
model = joblib.load(BytesIO(response.content))

In [ ]:
from ipywidgets import interact, Dropdown, IntSlider, FloatSlider
from google.colab import output  # Required for interactivity in Colab

# Enable interactive widgets in Colab
output.enable_custom_widget_manager()


# Define the prediction function
def price_predictor(suburb, area, latitude, longitude):
    df = pd.DataFrame(
        {
            "Suburb": [suburb],
            "BuildingArea": [area],
            "Latitude": [latitude],
            "Longitude": [longitude],
        }
    )
    df = df[model.feature_names_in_]  # Ensure correct feature order
    prediction = model.predict(df).round(2)[0]

    return f"💰 Predicted House Price: ${prediction}"



# Run interactive widget
widget = interact(
    price_predictor,
    suburb=Dropdown(options=sorted(X_train["Suburb"].unique()), description="Suburb:"),
    area=IntSlider(
        min=X_train["BuildingArea"].min(),
        max=X_train["BuildingArea"].max(),
        value=X_train["BuildingArea"].mean(),
        description="BuildingArea:",
    ),
    latitude=FloatSlider(
        min=X_train["Latitude"].min(),
        max=X_train["Latitude"].max(),
        step=0.01,
        value=X_train["Latitude"].mean(),
        description="Latitude:",
    ),
    longitude=FloatSlider(
        min=X_train["Longitude"].min(),
        max=X_train["Longitude"].max(),
        step=0.01,
        value=X_train["Longitude"].mean(),
        description="Longitude:",
    ),
)

interactive(children=(Dropdown(description='Suburb:', options=('Abbotsford', 'Aberfeldie', 'Airport West', 'Al…